<a href="https://colab.research.google.com/github/rrankawat/pytorch-cnn-mnist/blob/main/CNN_Using_ChatGPT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

In [2]:
# -------------------
# Device setup
# -------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
# -------------------
# Data loading
# -------------------
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

train_dataset = datasets.MNIST(root="./data", train=True, transform=transform, download=True)
test_dataset = datasets.MNIST(root="./data", train=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

100%|██████████| 9.91M/9.91M [00:00<00:00, 37.6MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 1.06MB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 9.14MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 7.21MB/s]


In [4]:
# -------------------
# CNN Model
# -------------------
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)   # (28x28 → 26x26)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)  # (26x26 → 24x24)
        self.fc1 = nn.Linear(9216, 128)       # flatten (64*12*12)
        self.fc2 = nn.Linear(128, 10)         # 10 classes

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)                # downsample (12x12)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

model = CNN().to(device)

In [5]:
# -------------------
# Optimizer & Loss
# -------------------
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [6]:
# -------------------
# Training function
# -------------------
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print(f"Train Epoch: {epoch} [{batch_idx * len(data)}/{len(train_loader.dataset)} "
                  f"({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}")

In [7]:
# -------------------
# Testing function
# -------------------
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction="sum").item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    print(f"\nTest set: Average loss: {test_loss:.4f}, Accuracy: {accuracy:.2f}%\n")

In [8]:
# -------------------
# Run Training
# -------------------
for epoch in range(1, 6):  # 5 epochs
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.294490
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.298726
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.072060
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.085314
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.111355
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.033061
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.061145
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.090535
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.072500
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.010585

Test set: Average loss: 0.0441, Accuracy: 98.59%

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.007890
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.051902
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.139384
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.014129
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.109972
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.071687
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.026159
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.016558
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.011496
Train Epoch